### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [6]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[20])


[...]

These don't seem like "little things" to me.  At least, they are orders
worse than the motto.  Do you think that the motto is a "little thing"
that will lead to worse things?


In [7]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [8]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [9]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [10]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [11]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [12]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [13]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [14]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [15]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [16]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750], dtype=int64)

In [19]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [20]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [21]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [22]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [23]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [24]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


### Respuestas

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

In [28]:
import random

In [29]:
def analizar_similitud_documentos(newsgroups_train, n_documentos=5, random_seed=42):

     # Se fija la semilla para poder repetir el experimento y ver los mismos documentos.
    random.seed(random_seed)
    np.random.seed(random_seed)

    # Convertimos los textos a vectores TF-IDF
    vectorizer = TfidfVectorizer(stop_words='english')
    X_train = vectorizer.fit_transform(newsgroups_train.data)
    
    # Seleccionamos n documentos al azar
    total_docs = len(newsgroups_train.data)
    docs_aleatorios = random.sample(range(total_docs), n_documentos)
    
    resultados = {}
    
    for idx in docs_aleatorios:
        # midamos la similaridad coseno con todos los documentos de train
        similitudes = cosine_similarity(X_train[idx:idx+1], X_train).flatten()
   
        # Ordenamos los índices por similitud (excluyendo el documento mismo)
        similitudes[idx] = -1  # Excluimos la auto-similitud
        indices_ordenados = np.argsort(similitudes)[::-1][:5]  # Top 5 más similares
        
        resultados[idx] = {
            'documento_original': newsgroups_train.data[idx][:200] + "...",  # Primeros 200 caracteres
            'categoria': newsgroups_train.target_names[newsgroups_train.target[idx]],
            'documentos_similares': [
                {
                    'indice': i,
                    'similitud': similitudes[i],
                    'categoria': newsgroups_train.target_names[newsgroups_train.target[i]],
                    'texto': newsgroups_train.data[i][:200] + "..."
                }
                for i in indices_ordenados
            ]
        }
    
    return resultados


In [30]:
resultados = analizar_similitud_documentos(newsgroups_train)

In [42]:
i=1
for doc_idx, info in resultados.items():
    print(f"\nDocumento original N°{doc_idx} (Categoría: {info['categoria']}):")
    print(info['documento_original'])
    print("\n***************** Documentos más similares: ****************")
    for doc in info['documentos_similares']:
        print(f"\nDOCUMENTO SIMILAR: {i}")
        print(f"\n\tÍndice: {doc['indice']}")
        print(f"\tSimilitud: {doc['similitud']:.4f}")
        print(f"\tCategoría: {doc['categoria']}")
        print(f"\tTexto: {doc['texto']}")
        i+=1
    print("-" * 80)
    i=1


Documento original N°10440 (Categoría: rec.sport.hockey):

These new rule changes are great!  However, I think that your rules are
MUCH too complicated.  How will the normal average fan be able to count
how many fouls a player has?  And then we would even ha...

***************** Documentos más similares: ****************

DOCUMENTO SIMILAR: 1

	Índice: 9434
	Similitud: 0.3907
	Categoría: rec.sport.baseball
	Texto: 

What makes you think Buck will still be in New York at year's end with
George back?  :-)

--
    Keith Keller				LET'S GO RANGERS!!!!!
						LET'S GO QUAKERS!!!!!
	kkeller@mail.sas.upenn.edu		IVY LE...

DOCUMENTO SIMILAR: 2

	Índice: 4837
	Similitud: 0.3807
	Categoría: rec.sport.hockey
	Texto: 
I think that they go to divisional records before goals, but I could be
wrong, too.

--
    Keith Keller				LET'S GO RANGERS!!!!!
						LET'S GO QUAKERS!!!!!
	kkeller@mail.sas.upenn.edu		IVY LEAGUE C...

DOCUMENTO SIMILAR: 3

	Índice: 1653
	Similitud: 0.3698
	Categoría: rec.sport.h

Resumen del punto 1:

Documento N°1:
	Tema: rec.sport.hockey

Documentos similares:  
1- Índice: 9434  
   Similitud: 0.3907  
   Categoría: rec.sport.baseball  
2- Índice: 10511  
   Similitud: 0.2546  
   Categoría: comp.graphics  
3- Índice: 10078  
   Similitud: 0.2370  
   Categoría: rec.motorcycles  
4- Índice: 7139  
   Similitud: 0.2290  
   Categoría: talk.politics.misc  
5- Índice: 9858  
   Similitud: 0.2205  
   Categoría: comp.os.ms-windows.misc  
   
   
Documento N°2:
	Tema: comp.os.ms-windows.misc
	
Documentos similares:  
1-  Índice: 346  
	Similitud: 0.3567  
	Categoría: comp.sys.ibm.pc.hardware  
2-  Índice: 3254  
	Similitud: 0.2951  
	Categoría: comp.graphics  
3- 	Índice: 5971  
	Similitud: 0.2947  
	Categoría: comp.sys.ibm.pc.hardware  
4- 	Índice: 6927  
	Similitud: 0.2742
	Categoría: comp.sys.ibm.pc.hardware  
5- 	Índice: 1782  
	Similitud: 0.2718  
	Categoría: comp.os.ms-windows.misc  
	
	
Documento N°3:
	Tema: misc.forsale	

1-  Índice: 765  
	Similitud: 0.2520  
	Categoría: comp.sys.mac.hardware  
2-  Índice: 7613  
	Similitud: 0.2418  
	Categoría: comp.sys.mac.hardware  
3-  Índice: 1083  
	Similitud: 0.2193  
	Categoría: comp.os.ms-windows.misc  
4-  Índice: 8141  
	Similitud: 0.2116  
	Categoría: sci.electronics  
5-  Índice: 11283  
	Similitud: 0.1980  
	Categoría: comp.os.ms-windows.misc  


Documento N°4:
	Tema: sci.electronics

1- Índice: 5571  
	Similitud: 0.1306  
	Categoría: misc.forsale  
2- 	Índice: 428  
	Similitud: 0.1086  
	Categoría: comp.sys.mac.hardware  
3- 	Índice: 3140  
	Similitud: 0.1086  
	Categoría: comp.sys.mac.hardware  
4- 	Índice: 9266  
	Similitud: 0.1086  
	Categoría: rec.motorcycles  
5- 	Índice: 7811  
	Similitud: 0.1072  
	Categoría: comp.sys.ibm.pc.hardware  
	
Documento N°5:
	Tema: comp.graphics
	
1-  Índice: 8270  
	Similitud: 0.3771  
	Categoría: comp.os.ms-windows.misc  
2-  Índice: 10511  
	Similitud: 0.2546  
	Categoría: comp.graphics  
3-  Índice: 10078  
	Similitud: 0.2370  
	Categoría: rec.motorcycles  
4-  Índice: 7139  
	Similitud: 0.2290  
	Categoría: talk.politics.misc  
5-  Índice: 9858  
	Similitud: 0.2205  
	Categoría: comp.os.ms-windows.misc  

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [46]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score

def entrenar_y_evaluar_modelos(X_train, y_train, X_test, y_test):

    # Pipelines para cada tipo de modelo
    pipeline_mnb = Pipeline([
        ('vectorizer', TfidfVectorizer()),
        ('classifier', MultinomialNB())
    ])

    pipeline_cnb = Pipeline([
        ('vectorizer', TfidfVectorizer()),
        ('classifier', ComplementNB())
    ])

    # Set de parámetros para rpobar con GridSesarch 
    parametros = {
        'vectorizer__max_features': [None, 10000, 50000], # Prueba con todaslas palabras, con las 10000 mas frecuentes o con las 50000 mas frecuentes
        'vectorizer__ngram_range': [(1,1), (1,2)], # Pureba con unigramas y bigramas
        'vectorizer__min_df': [1, 2, 5], # Ignora términos que tenga un porcentaje de ocurrencia menor a este
        'vectorizer__max_df': [0.95, 0.9, 0.8], # Ignora términos que tenga un porcentaje de ocurrencia mayor a este
    }

    # Entrena modelos
    resultados = {}
    for nombre, pipeline in [('MultinomialNB', pipeline_mnb), ('ComplementNB', pipeline_cnb)]:
        print(f"\nEntrenando {nombre}...")
        
        # Búsqueda de mejores parámetros usando validación cruzada
        grid_search = GridSearchCV(
            pipeline,
            parametros,
            cv=5,
            scoring='f1_macro',
            n_jobs=-1,
            verbose=1
        )
        
        grid_search.fit(X_train, y_train)
        
        # Resultados
        resultados[nombre] = {
            'mejor_score_cv': grid_search.best_score_,
            'mejores_params': grid_search.best_params_,
            'mejor_modelo': grid_search.best_estimator_
        }
        
        # Evaluamos en conjunto de test
        y_pred = grid_search.predict(X_test)
        f1 = f1_score(y_test, y_pred, average='macro')
        
        print(f"\nMejores parámetros para {nombre}:")
        print(grid_search.best_params_)
        print(f"\nF1-score (macro) en validación cruzada: {grid_search.best_score_:.4f}")
        print(f"F1-score (macro) en test: {f1:.4f}")
        print("\nReporte de clasificación detallado:")
        print(classification_report(y_test, y_pred))
        
    return resultados

def analizar_errores(modelo, X_test, y_test, target_names):

    y_pred = modelo.predict(X_test)
    
    # Encontrar ejemplos mal clasificados
    errores_indices = np.where(y_pred != y_test)[0]
    
    errores_analisis = []
    for idx in errores_indices[:10]:  # Analizamos los primeros 10 errores
        errores_analisis.append({
            'texto': X_test[idx][:200] + "...",  # Primeros 200 caracteres
            'categoria_real': target_names[y_test[idx]],
            'categoria_predicha': target_names[y_pred[idx]],
            'probabilidades': modelo.predict_proba([X_test[idx]])[0]
        })
    
    return errores_analisis


In [47]:
# Entrenamos y evaluamos los modelos
resultados = entrenar_y_evaluar_modelos(
    newsgroups_train.data,
    newsgroups_train.target,
    newsgroups_test.data,
    newsgroups_test.target
)

# Analizamos los errores del mejor modelo
mejor_modelo = max(resultados.items(), key=lambda x: x[1]['mejor_score_cv'])
print(f"\nAnálisis de errores para el mejor modelo ({mejor_modelo[0]}):")
errores = analizar_errores(
    mejor_modelo[1]['mejor_modelo'],
    newsgroups_test.data,
    newsgroups_test.target,
    newsgroups_test.target_names
)

# Mostramos algunos ejemplos de errores
for i, error in enumerate(errores, 1):
    print(f"\nError #{i}")
    print(f"Texto: {error['texto']}")
    print(f"Categoría real: {error['categoria_real']}")
    print(f"Categoría predicha: {error['categoria_predicha']}")
    print("Top 3 probabilidades:")
    probs_ordenadas = sorted(enumerate(error['probabilidades']), key=lambda x: x[1], reverse=True)[:3]
    for idx, prob in probs_ordenadas:
        print(f"- {newsgroups_test.target_names[idx]}: {prob:.4f}")


Entrenando MultinomialNB...
Fitting 5 folds for each of 54 candidates, totalling 270 fits

Mejores parámetros para MultinomialNB:
{'vectorizer__max_df': 0.8, 'vectorizer__max_features': 10000, 'vectorizer__min_df': 5, 'vectorizer__ngram_range': (1, 1)}

F1-score (macro) en validación cruzada: 0.6533
F1-score (macro) en test: 0.6182

Reporte de clasificación detallado:
              precision    recall  f1-score   support

           0       0.67      0.15      0.25       319
           1       0.64      0.67      0.66       389
           2       0.66      0.55      0.60       394
           3       0.57      0.72      0.64       392
           4       0.77      0.61      0.68       385
           5       0.79      0.75      0.77       395
           6       0.82      0.74      0.78       390
           7       0.76      0.70      0.73       396
           8       0.81      0.73      0.77       398
           9       0.90      0.76      0.82       397
          10       0.58      0.89

## Resumen:

Modelo: MultinomialNB

Mejores parámetros
'vectorizer__max_df': 0.8, 'vectorizer__max_features': 10000, 'vectorizer__min_df': 5, 'vectorizer__ngram_range': (1, 1)}

Iterpretación:
'vectorizer__max_df': 0.8 ==> Se eliminan las palabras muy frecuentes. Debe ser porque si se repiten tanto, no deben ser tan importantes
'vectorizer__min_df': 5 ==> Se eliminan las palabras que tienen poca repitencia.
'vectorizer__max_features': 10000 ==> Prueba con las 10000 mas frecuentes
'vectorizer__ngram_range': (1, 1) ==> La mejor opcion es con unigramas. Parece que permite desmenusar mejor las palabras

F1-score (macro) en validación cruzada: 0.6533
F1-score (macro) en test: 0.6182

El modelo tiene algo de overfiting, ya que la métrica de validación tiene un valor inferior a la test

-------------------------------------------------------------------------------------------------------------------------

Modelo: ComplementNB

Mejores parámetros
{'vectorizer__max_df': 0.8, 'vectorizer__max_features': 50000, 'vectorizer__min_df': 1, 'vectorizer__ngram_range': (1, 1)}

Iterpretación:
'vectorizer__max_df': 0.8 ==> Se eliminan las palabras muy frecuentes. Debe ser porque si se repiten tanto, no deben ser tan importantes
'vectorizer__min_df': 1 ==> Se eliminan las palabras que tienen poca repitencia. En este caso es mas estricto que el modelo MultinomialNB
'vectorizer__max_features': 50000 ==> Prueba con las 50000 mas frecuentes
'vectorizer__ngram_range': (1, 1) ==> La mejor opcion es con unigramas. Parece que permite desmenusar mejor las palabras

F1-score (macro) en validación cruzada: 0.7535
F1-score (macro) en test: 0.6933

El modelo performa mejor que MultinomialNB, pero se observa tambien overfiting.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [49]:
import pandas as pd

def analizar_similitud_palabras(newsgroups_train, palabras_interes):

    # Se instancia un vectorizador TfidfVectorizer conb los hiperparámetros obtenidos en el punto anterior
    vectorizer = TfidfVectorizer(max_df=0.8, min_df=5, max_features=10000    )
    
    # Matriz documento-término
    X = vectorizer.fit_transform(newsgroups_train.data)
    
    # Obtenemos el vocabulario y la matriz transpuesta (término-documento)
    vocabulary = vectorizer.get_feature_names_out()
    X_transpuesta = X.T
    
    # Verificamos que las palabras de interés estén en el vocabulario
    palabras_validas = [palabra for palabra in palabras_interes if palabra in vectorizer.vocabulary_]
    palabras_no_encontradas = set(palabras_interes) - set(palabras_validas)
    
    if palabras_no_encontradas:
        print(f"Advertencia: Las siguientes palabras no se encontraron en el vocabulario: {palabras_no_encontradas}")
    
    resultados = {}
    
    for palabra in palabras_validas:
        # Obtenemos el índice de la palabra en el vocabulario
        idx_palabra = vectorizer.vocabulary_[palabra]
        
        # Calculamos similitud con todas las demás palabras
        vector_palabra = X_transpuesta[idx_palabra].toarray()
        similitudes = cosine_similarity(vector_palabra, X_transpuesta.toarray())[0]
        
        # Ordenamos las palabras por similitud (excluyendo la palabra misma)
        similitudes[idx_palabra] = -1
        indices_top = similitudes.argsort()[::-1][:5]
        
        # Guardamos las palabras más similares y sus puntajes
        palabras_similares = [
            (vocabulary[idx], similitudes[idx])
            for idx in indices_top
        ]
        
        resultados[palabra] = palabras_similares
        
        # Imprimimos los resultados
        print(f"\nPalabra: {palabra}")
        for palabra_similar, score in palabras_similares:
            print(f"- {palabra_similar}: {score:.4f}")
    
    return resultados

# Lista de palabras elegidas manualmente. Como el diccionario está en inglés, las palabras se colocan en inglés.
palabras_interes = ['science', 'tree', 'clothes','vacation','electronics']

# Ejecutamos el análisis
resultados = analizar_similitud_palabras(newsgroups_train, palabras_interes)



Palabra: science
- scientific: 0.3336
- empirical: 0.2737
- hypotheses: 0.1890
- fiction: 0.1842
- scientists: 0.1781

Palabra: tree
- immaculate: 0.2701
- righteous: 0.2575
- palm: 0.2472
- u2: 0.2131
- christmas: 0.2078

Palabra: clothes
- whites: 0.3479
- cramer: 0.2502
- coat: 0.2463
- clothing: 0.2445
- substitute: 0.2373

Palabra: vacation
- las: 0.3774
- orlando: 0.2984
- vegas: 0.2732
- hotel: 0.2666
- trip: 0.2333

Palabra: electronics
- 805: 0.1738
- addressed: 0.1646
- diagram: 0.1627
- mq: 0.1520
- females: 0.1477


## Resumen:

Las palabras elegidas fueron: science, tree, clothes, vacation, electronics


Palabra: science
Las 5 más similares:  
- scientific: 0.3336  
- empirical: 0.2737  
- hypotheses: 0.1890  
- fiction: 0.1842  
- scientists: 0.1781  

Observaciones: Todas las palabras tienen alta relación con la palabra buscada

Palabra: tree  
Las 5 más similares:  
- immaculate: 0.2701  
- righteous: 0.2575  
- palm: 0.2472  
- u2: 0.2131  
- christmas: 0.2078  

Observaciones: Acá se ve confusión.    
	immaculate - No sé que relación encuentra    
	righteous - No sé que relación encuentra    
	palm - Palmera, es una variedad de árbol    
	u2 - Supongo que la banda musical. No sé que relación encuentra    
	christmas - Relacionada por àrbol de navidad    

Palabra: clothes  
Las 5 más similares:  
- whites: 0.3479  
- cramer: 0.2502  
- coat: 0.2463  
- clothing: 0.2445  
- substitute: 0.2373  

Observaciones: Algunas bien, algunas mal  
	whites - (BLancas) Tiene relación con la palabra ropa  
	cramer - No se que significa  
	coat - Campera/abrigo, tiene relación  
	clothing - Sinónimo de ropa  
	substitute - No sé que relación encuentra  

Palabra: vacation  
Las 5 más similares:  
- las: 0.3774  
- orlando: 0.2984  
- vegas: 0.2732  
- hotel: 0.2666  
- trip: 0.2333  

Observaciones: 	Algunas bien, algunas mal  
	las: Supongo que lo relaciona con Las Vegas, esta bien pero corta la palabra  
	orlando: Correcto  
	vegas: Supongo que lo relaciona con Las Vegas, esta bien pero corta la palabra  
	hotel: Correcto  
	trip: Correcto  

Palabra: electronics  
Las 5 más similares:  
- 805: 0.1738  
- addressed: 0.1646  
- diagram: 0.1627  
- mq: 0.1520  
- females: 0.1477  

Observaciones: 	Algunas bien, algunas mal  

	805: No sé que relación encuentra  
	addressed: Tiene relación  
	diagram: Tiene relación  
	mq: No sé que significa "mq"   
	females: No sé que relación encuentra  
